# Multi-robot Task Allocation

In [48]:
using Symbolics
using Test
using Plots
using Latexify
using LaTeXStrings

LoadError: ArgumentError: Package Latexify not found in current path.
- Run `import Pkg; Pkg.add("Latexify")` to install the Latexify package.

In [46]:
function forward_pose(r)
    x₁, y₁, θ₁, x₂, y₂, θ₂ = r

    a = x₁ - x₂
    b = y₁ - y₂

    xᶜ = (x₁ + x₂) / 2
    yᶜ = (y₁ + y₂) / 2
    d = (1 / 2) * sqrt(a^2 + b^2)

    θᶜ = atan(a,b)

    ϕ₁ = θ₁ - θᶜ
    ϕ₂ = θ₂ - θᶜ
    
    return xᶜ, yᶜ, d, θᶜ, ϕ₁, ϕ₂
end

function inverse_pose(c)
    xᶜ, yᶜ, d, θᶜ, ϕ₁, ϕ₂ = c

    x₁ = xᶜ + d * sin(θᶜ)
    x₂ = xᶜ - d * sin(θᶜ)

    y₁ = yᶜ + d * cos(θᶜ)
    y₂ = yᶜ - d * cos(θᶜ)

    θ₁ = θᶜ + ϕ₁
    θ₂ = θᶜ + ϕ₂

    return x₁, y₁, θ₁, x₂, y₂, θ₂
end;

In [47]:
function test_forward_pose()
    robots_pose = [3.5, -2.2, 1.0, 0.0, 3.4, 33.3]

    cluster_pose = forward_pose(robots_pose)

    @test all(isapprox.(cluster_pose, [1.75, 0.6, 3.3018, 2.5829, -1.5829, 30.7170], atol=1e-3))
end

function test_inverse_pose()
    cluster_pose = [1.75, 0.6, 3.3018, 2.5829, -1.5829, 30.7170]
    robots_pose = inverse_pose(cluster_pose)

    @test all(isapprox.(robots_pose, [3.5, -2.2, 1.0, 0.0, 3.4, 33.3], atol=1e-3))
end

test_forward_pose()
test_inverse_pose()

Test Passed

In [59]:
@variables x₁ x₂ x₃ y₁ y₂ y₃ α

7-element Vector{Num}:
 x₁
 x₂
 x₃
 y₁
 y₂
 y₃
  α

In [60]:
β = atan((x₃-x₁)*sin(α) + (y₃ - y₁)*cos(α), (x₃-x₁)*cos(α) - (y₃-y₁) * sin(α))

atan((-x₁ + x₃)*sin(α) + (-y₁ + y₃)*cos(α), (-x₁ + x₃)*cos(α) - (-y₁ + y₃)*sin(α))

In [63]:
D = Differential(x₁)
expand_derivatives(D(β))

(-((-x₁ + x₃)*cos(α) - (-y₁ + y₃)*sin(α))*sin(α)) / (((-x₁ + x₃)*cos(α) - (-y₁ + y₃)*sin(α))^2 + ((-x₁ + x₃)*sin(α) + (-y₁ + y₃)*cos(α))^2) + (-(-(-x₁ + x₃)*sin(α) - (-y₁ + y₃)*cos(α))*cos(α)) / (((-x₁ + x₃)*cos(α) - (-y₁ + y₃)*sin(α))^2 + ((-x₁ + x₃)*sin(α) + (-y₁ + y₃)*cos(α))^2)